In [2]:
import spacy
from spacy.matcher import Matcher

## Introduccion

### Pipeline Basico

In [3]:
nlp = spacy.blank("es")

/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
doc = nlp("Hola mundo!")

for token in doc:
    print(token.text)

Hola
mundo
!


### Pipeline Large

##### Etiquetado Gramatical

In [5]:
nlp = spacy.load("es_core_news_lg")

doc = nlp("Ella comió pizza")

for token in doc:
    print(token.text, token.pos_)

Ella PRON
comió VERB
pizza PROPN


##### NER

In [6]:
# Procesa un texto
doc = nlp(
    "Apple es la marca que más satisfacción genera en EE.UU., "
    "pero el iPhone, fue superado por el Galaxy Note 9"
)

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(ent.text, ent.label_)

Apple ORG
EE.UU. LOC
iPhone MISC
Galaxy Note 9 MISC


In [7]:
# Procesa un texto
doc = nlp(
    "River Plate es el equipo mas grande de Argentina "
)

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(ent.text, ent.label_)

River Plate ORG
Argentina LOC


In [8]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

#### Pattern Maching

In [9]:
matcher = Matcher(nlp.vocab)

pattern = [{"TEXT": "adidas"}, {"TEXT": "zx"}]
matcher.add("ADIDAS_PATTERN", [pattern])

doc = nlp("Nuevos diseños de zapatillas en la colección adidas zx")

matches = matcher(doc)

for match_id, start, end in matches:
    # Obtén el span resultante
    matched_span = doc[start:end]
    print(matched_span.text)

adidas zx


In [10]:
pattern = [
    {"IS_DIGIT": True},
    {"LOWER": "copa"},
    {"LOWER": "mundial"},
    {"LOWER": "fifa"},
    {"IS_PUNCT": True}
]
doc = nlp("2014 Copa Mundial FIFA: Alemania ganó!")


In [11]:
pattern = [
    {"LEMMA": "comer", "POS": "VERB"},
    {"POS": "NOUN"}
]
doc = nlp("Camila prefería comer sushi. Pero ahora está comiendo pasta.")


### Analisis de Datos

#### Hashes

In [12]:
doc = nlp("Yo tengo un gato")

# Busca el hash para la palabra "gato"
gato_hash = nlp.vocab.strings["gato"]
print(gato_hash)

# Busca el gato_hash para obtener el string
gato_string = nlp.vocab.strings[gato_hash]
print(gato_string)

9565357104409163886
gato


#### Doc & Span

In [13]:
from spacy.tokens import Doc, Span

# Las palabras y espacios que usaremos para crear el doc
words = ["¡", "Hola", "Mundo", "!"]
spaces = [False, True, False, False]

# Crea un doc manualmente
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Crea un span manualmente
span = Span(doc, 1, 3)

# Crea un span con un label
span_with_label = Span(doc, 1, 3, label="SALUDO")

# Añade el span a los doc.ents
doc.ents = [span_with_label]

#### Word2Vec

In [14]:
doc1 = nlp("Me gusta la comida rápida")
doc2 = nlp("Me gusta la pizza")
print(doc1.similarity(doc2))

0.9513663710080219


In [15]:
doc = nlp("Me gusta la pizza")
token = nlp("jabón")[0]

print(doc.similarity(token))

0.1363773817075813


In [16]:
doc = nlp("Tengo una manzana")
# Accede al vector a través del atributo token.vector
print(doc[2].vector)

[-0.58136    0.037496   0.66934    2.7966    -0.023352   0.39145
  0.55106    0.2597     2.6257     3.1932    -0.49274    0.084971
  0.083045  -1.1788    -0.11184    0.052108  -0.563      0.2155
 -1.5244    -1.9768    -1.6693    -0.85393    0.89011   -0.99332
  1.7136    -1.7498    -1.5536     0.44981    0.76886    1.298
  0.094683  -0.07842    1.1843    -1.5305    -0.44663    1.3727
  1.2239    -1.4967     0.75916    0.70923    1.4964     0.56073
 -1.6018    -0.91338   -2.0583     1.1208    -0.86255    0.76231
  0.60926   -1.0933    -2.0223    -1.2327     0.24917    0.95122
 -1.0975    -0.83044   -1.4911    -0.79706   -0.23831    0.10205
 -0.44532    1.0172     1.2452    -2.0414    -0.39335    1.149
 -0.0094314  1.5695    -2.2982     1.2705     0.59918    0.95636
  0.20392    0.35687    1.6167    -0.80719   -0.52339    0.68923
 -0.3944    -3.0173     1.0634    -3.4248    -0.29587   -1.2832
 -1.6083     0.74691   -0.11828    1.4702     0.16136   -1.4201
  0.70638   -0.072624  -1.4466  

In [17]:
doc1 = nlp("Me gustan los gatos")
doc2 = nlp("Me desagradan los gatos")

print(doc1.similarity(doc2))

0.9761564889481511


#### Phrase Matcher

In [18]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("labrador dorado")
matcher.add("PERRO", [pattern])
doc = nlp("Tengo un labrador dorado")

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Obtén el span resultante
    span = doc[start:end]
    print("span resultante:", span.text)

span resultante: labrador dorado


### Pipelines

In [19]:
nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [20]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f294c0c8d70>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f294743e090>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f2947296ea0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f294701a2d0>),
 ('lemmatizer',
  <spacy.lang.es.lemmatizer.SpanishLemmatizer at 0x7f29472f8250>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f2946f87df0>)]

#### Components

In [25]:
from spacy.language import Language

In [24]:
@Language.component("custom_component")
def custom_component_function(doc):
    # Imprime la longitud del doc en pantalla
    print("longitud del Doc:", len(doc))
    # Devuelve el objeto doc
    return doc

# Añade el componente al primer lugar del pipeline
nlp.add_pipe("custom_component", first=True)

# Imprime los nombres de los componentes del pipeline
print("Pipeline:", nlp.pipe_names)

Pipeline: ['custom_component', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [28]:
doc = nlp("¡Hola Mundo!")

longitud del Doc: 4


En este ejercicio escribirás un componente personalizado que use el PhraseMatcher para encontrar nombres de animales en el documento y añada los spans resultantes a los doc.ents. En la variable matcher ya se creó un PhraseMatcher con los patrones de animales.

Define el componente personalizado y aplica el matcher al doc.
Crea un Span para cada resultado, asígnale el label ID para "ANIMAL" y sobrescribe los doc.ents con los nuevos spans.
Añade el nuevo componente al pipeline después del componente "ner".
Procesa el texto e imprime en pantalla el texto de la entidad y los entity labels de las entidades en doc.ents.

In [29]:
animals = ["labrador dorado", "gato", "tortuga", "oso de anteojos"]
animal_patterns = list(nlp.pipe(animals))
print("patrones_de_animales:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", None, *animal_patterns)

# Define el componente personalizado
@Language.component("animal_component")
def animal_component_function(doc):
    # Aplica el matcher al doc
    matches = matcher(doc)
    # Crea un Span para cada resultado y asígnales el label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Sobrescribe los doc.ents con los spans resultantes
    doc.ents = spans
    return doc


# Añade el componente al pipeline después del componente "ner"
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# Procesa el texto e imprime en pantalla el texto y el label
# de los doc.ents
doc = nlp("Hoy vimos una tortuga y un oso de anteojos en nuestra caminata")
print([(ent.text, ent.label_) for ent in doc.ents])

longitud del Doc: 2
longitud del Doc: 1
longitud del Doc: 1
longitud del Doc: 3
patrones_de_animales: [labrador dorado, gato, tortuga, oso de anteojos]
['custom_component', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'animal_component']
longitud del Doc: 12
[('tortuga', 'ANIMAL'), ('oso de anteojos', 'ANIMAL')]


#### Extension de Atributos


In [30]:
# Importa las clases globales
from spacy.tokens import Doc, Token, Span

# Añade extensiones para el Doc, Token y Span
Doc.set_extension("title", default=None)
Token.set_extension("is_color", default=False)
Span.set_extension("has_color", default=False)

In [31]:
doc._.title = "Mi documento"
token._.is_color = True
span._.has_color = False

In [33]:
# Define la función getter
def get_is_color(token):
    colors = ["rojo", "amarillo", "azul"]
    return token.text in colors

# Añade una extensión en el Token con getter
Token.set_extension("is_color", getter=get_is_color,force=True)

doc = nlp("El cielo es azul.")
print(doc[3]._.is_color, "-", doc[3].text)

longitud del Doc: 5
True - azul


In [35]:
# Define la función getter
def get_has_color(span):
    colors = ["rojo", "amarillo", "azul"]
    return any(token.text in colors for token in span)

# Añade una extensión en el Span con getter
Span.set_extension("has_color", getter=get_has_color,force=True)

doc = nlp("El cielo es azul.")
print(doc[1:4]._.has_color, "-", doc[1:4].text)
print(doc[0:2]._.has_color, "-", doc[0:2].text)

longitud del Doc: 5
True - cielo es azul
False - El cielo


#### Extension de Metodos

In [36]:
# Define un método con argumentos
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# Añade una extensión en el Doc con el método
Doc.set_extension("has_token", method=has_token)

doc = nlp("El cielo es azul.")
print(doc._.has_token("azul"), "- azul")
print(doc._.has_token("nube"), "- nube")

longitud del Doc: 5
True - azul
False - nube


In [37]:
nlp_blank = spacy.blank("es")

# Define la función getter que toma un token y devuelve su texto al revés
def get_reversed(token):
    return token.text[::-1]


# Registra la extensión de propiedad del Token, "reversed", con
# el getter get_reversed
Token.set_extension("reversed", getter=get_reversed)

# Procesa el texto e imprime en pantalla el atributo "reversed"
# para cada token
doc = nlp_blank("Todas las generalizaciones son falsas, incluyendo esta.")
for token in doc:
    print("invertido:", token._.reversed)

invertido: sadoT
invertido: sal
invertido: senoicazilareneg
invertido: nos
invertido: saslaf
invertido: ,
invertido: odneyulcni
invertido: atse
invertido: .


En este ejercicio combinarás la extensión de atributos personalizados con las predicciones del modelo y crearás un getter de atributo que devuelve una URL de búsqueda de Wikipedia si el span es una persona, organización o lugar.

Completa el getter get_wikipedia_url para que solo devuelva la URL si el label del span está en la lista de labels.
Añade la extensión del Span, "wikipedia_url", usando el getter get_wikipedia_url.
Itera sobre las entidades en el doc y devuelve sus URLs de Wikipedia.

In [38]:
nlp = spacy.load("es_core_news_lg")


def get_wikipedia_url(span):
    # Obtén la URL de Wikipedia si el span tiene uno de los siguientes labels
    if span.label_ in ("PER", "ORG", "LOC"):
        entity_text = span.text.replace(" ", "_")
        return "https://es.wikipedia.org/w/index.php?search=" + entity_text


# Añade la extensión del Span, wikipedia_url, usando el getter get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "Antes de finalizar 1976, el interés de David Bowie en la "
    "floreciente escena musical alemana, le llevó a mudarse a "
    "Alemania para revitalizar su carrera."
)
for ent in doc.ents:
    # Imprime en pantalla el texto y la URL de Wikipedia de la entidad
    print(ent.text, ent._.wikipedia_url)

David Bowie https://es.wikipedia.org/w/index.php?search=David_Bowie
Alemania https://es.wikipedia.org/w/index.php?search=Alemania


### Recomendaciones

#### Pasar contexto

In [39]:
data = [
    ("Esto es un texto", {"id": 1, "numero_pagina": 15}),
    ("y otro texto", {"id": 2, "numero_pagina": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    print(doc.text, context["numero_pagina"])

Esto es un texto 15
y otro texto 16


In [40]:
Doc.set_extension("id", default=None)
Doc.set_extension("numero_pagina", default=None)

data = [
    ("Esto es un texto", {"id": 1, "numero_pagina": 15}),
    ("y otro texto", {"id": 2, "numero_pagina": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    doc._.id = context["id"]
    doc._.numero_pagina = context["numero_pagina"]

#### Usar solo el tokenizer si es necesario

In [41]:
doc = nlp.make_doc("¡Hola Mundo!")


#### Deshabilitar pipes

In [45]:
with nlp.select_pipes(disable=["senter", "parser"]):
    # Procesa el texto e imprime las entidades en pantalla
    doc = nlp("text")
    print(doc.ents)

()


#### Desarrollo nuevo modelo

In [46]:
import random
docs= ["Hola","Mundo"]
random.shuffle(docs)
train_docs = docs[:len(docs) // 2]
dev_docs = docs[len(docs) // 2:]

In [ ]:
from spacy.tokens import Span, DocBin
# Crea y guarda una colección de docs para entrenamiento
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("./train.spacy")
# Crea y guarda una colección de docs para evaluación
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("./dev.spacy")

##### Generar una configuración
- spaCy puede auto-generar un archivo de configuración default por ti
- interactivo wodget de inicio en los archivos
- init config comando en CLI
```
 python -m spacy init config ./config.cfg --lang es --pipeline ner
```
- init config: el comando a correr
- config.cfg: ruta de salida de la configuración generada
- --lang: la clase del idioma del pipeline, p. ej. es para español
- --pipeline: nombres, separados con comas, de los componentes a incluir

python -m spacy init config ./config.cfg --lang es --pipeline textcat --optimize accuracy spacy_categorization_pipeline.cfg

##### Entrenar un Pipeline
- todo lo que necesitas es el config.cfg y los datos de entrenamiento y desarrollo
- los ajustes de configuración pueden modificarse en la línea de comando
```
$ python -m spacy train ./config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy
```
- train: el comando a correr
- config.cfg: ruta de archivo de configuración
- --output: ruta del directorio de salida para guardar el pipeline entrenado
- --paths.train: anular con ruta a los datos de entrenamiento
- --paths.dev: anular con ruta a los datos de evaluación

##### Cargando un pipeline entrenado

- la salida de datos después de un pipeline normal de spaCy
- model-last: el pipeline entrenado más recientemente
- model-best: el pipeline mejor entrenado
- cárgalo con spacy.load
```
import spacy

nlp = spacy.load("/ruta/de/arhcivo/model-best")
doc = nlp("iPhone 11 vs iPhone 8: Cuál es la diferencia?")
print(doc.ents)
```

##### Empaquetando tu pipeline

- spacy package: crea un paquete de Python instalable que contiene tu pipeline
- es fácil de versionar y de 
```
$ python -m spacy package /path/to/output/model-best ./packages --name mi_pipeline --version 1.0.0
```
```
$ cd ./packages/es_mi_pipeline-1.0.0
$ pip install dist/es_mi_pipeline-1.0.0.tar.gz
```
- Carga y utiliza el pipeline después de instalarlo:
- nlp = spacy.load("es_mi_pipeline")